In [ ]:
import nibabel as nib
from nibabel.testing import data_path

import os
from pathlib import Path 

import math
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure